# Handwritten numbers Recognition

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 

## Load Data

Extraer datos. Las imágenes son de 28*28 píxeles, por lo tanto el csv tiene 784 columnas más 1 indicando la salida

In [18]:
def extract():
    df_train = pd.read_csv('data/mnist_train.csv', delimiter = ',').to_numpy()
    df_test = pd.read_csv('data/mnist_test.csv', delimiter = ',').to_numpy()
    df = pd.DataFrame(np.concatenate((df_train,df_test)))
    return df

Transformar output a array de 10 y separar de las inputs

In [19]:
def transform(df): 
    y_raw = df.iloc[:, 0].to_numpy()
    x = df.drop(df.columns[0], axis='columns').to_numpy()
    y = []
    for i in range(0,y_raw.shape[0]):
        row=np.zeros(10)
        row[y_raw[i]]=1
        y.append(row)
    y=np.array(y)
    y.reshape(y.shape[0],10)
    return x, y

Creamos los dataset de entrenamiento y test

In [20]:
df = extract()
x_train,y_train = transform(df.iloc[0:60000])
#x_val,y_val = transform(df.iloc[50000:60000])
x_test,y_test = transform(df.iloc[60000:70000])

## Creación de la Red Neuronal

En la red neuronal vamoas a implementar una arquitectura de (784,512,256,128,10), con funciones de activación ReLu en todas las capas menos en la última, en la cual utilizaremos la funcion softmax (https://www.quora.com/Why-is-it-better-to-use-Softmax-function-than-sigmoid-function).<br> Para cada capa se utilizará la funcion Dense, la cual tiene los siguientes parámetros por defecto:
- activation=None,
- use_bias=True,
- kernel_initializer='glorot_uniform',
- bias_initializer='zeros',
- kernel_regularizer=None,
- bias_regularizer=None,
- activity_regularizer=None,
- kernel_constraint=None,
- bias_constraint=None




In [21]:
model = keras.Sequential([
    keras.layers.Dense(784, input_dim=784),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")#Softmax is a sort of Multi Class Sigmoid
])

Vamos a crear y entrenar el modelo, 50 epochs serán suficientes.

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 7s 112us/sample - loss: 0.0862 - accuracy: 0.9886
Epoch 2/50
60000/60000 [==============================] - 6s 104us/sample - loss: 0.0609 - accuracy: 0.9885
Epoch 3/50
60000/60000 [==============================] - 7s 109us/sample - loss: 0.1227 - accuracy: 0.9816
Epoch 4/50
60000/60000 [==============================] - 6s 108us/sample - loss: 0.0645 - accuracy: 0.9866
Epoch 5/50
60000/60000 [==============================] - 7s 112us/sample - loss: 0.1155 - accuracy: 0.9765
Epoch 6/50
60000/60000 [==============================] - 7s 109us/sample - loss: 0.1345 - accuracy: 0.9725
Epoch 7/50
60000/60000 [==============================] - 7s 110us/sample - loss: 0.0860 - accuracy: 0.9812
Epoch 8/50
60000/60000 [==============================] - 7s 110us/sample - loss: 0.0542 - accuracy: 0.9888
Epoch 9/50
60000/60000 [==============================] - 7s 111us/sample - loss: 0.0531 - accuracy: 0.9886
Epoch

Evaluamos el modelo con el acierto de los datos de prueba

In [23]:
tes_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 132us/sample - loss: 0.1259 - accuracy: 0.9786
Test Acc:  0.97859573
